<a href="https://colab.research.google.com/github/snaiws/Project_5/blob/main/API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [93]:
#라이브러리
import requests
from urllib import parse
import pandas as pd
import time
from IPython.display import display, clear_output #ipynb에서 api key 안보이게 하기 위해

In [ ]:
#api시간제한 << 자동으로 되나??


In [63]:
#버전,티어별 닉넴 뽑기
def getSummonerName(tier,page,KEY): # II구간 get
  APIURL = f"https://kr.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{tier}/II?page={page}&api_key={KEY}"
  res = requests.get(APIURL)
  data = res.json()
  data = [x['summonerName'] for x in data if x['inactive'] == False]
  return data

In [60]:
#get puuid
def getPuuid(summonerName,KEY): #나중에 쓸 일 있을 수도 있으니 DB로 저장
  encodedSummonerName = parse.quote(summonerName)
  APIURL = f"https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/{encodedSummonerName}?api_key={KEY}"
  res = requests.get(APIURL)
  data = res.json()
  return data['puuid']

In [61]:
#get MatchId
def getMatchId(puuid,KEY): #나중에 쓸 일 있을 수도 있으니 DB로 저장
  APIURL = f"https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type=ranked&start=0&count=30&api_key={KEY}"
  res = requests.get(APIURL)
  data = res.json()
  return data

In [86]:
#getMatch
def getMatch(matchId,KEY): #나중에 쓸 일 있을 수도 있으니 DB로 저장
  APIURL = f"https://asia.api.riotgames.com/lol/match/v5/matches/{matchId}?api_key={KEY}"
  res = requests.get(APIURL)
  data = res.json()
  version = data['info']['gameVersion']
  data = data['info']['participants']
  data = [[x['individualPosition'],x['championName'],x['win']] for x in data]
  return version, data

In [ ]:
def getData(tier,version,KEY):
  df = pd.DataFrame()
  while len(df) < 50000:
  summonerList = getSummonerName('DIAMOND','1',KEY)
  summonerList = [getPuuid(x,KEY) for x in summonerList]
  matchList = []
  for i in summonerList:
    matchList = matchList + getMatchId(i,KEY)
  matchList = list(set(matchList))
  for i in matchList:
    getMatch(matchId,KEY)


In [99]:
KEY = input()
clear_output(wait=True)
True

True

In [ ]:
#최신버전 불러오기
def getRecentVersion(KEY):
  